Check Duplicates and drop / Check each labels got how many data

In [ ]:
import pandas as pd

# Load the data from the CSV file
file_path = "Compiled_ALL_Cryptocurrency.csv"
data = pd.read_csv(file_path)

# Remove duplicate rows based on the "text" column
data.drop_duplicates(subset="text", keep="first", inplace=True)

# Select specific columns
data = data[["text", "annotation"]]

# Drop rows with any missing values
data = data.dropna()

# Save the DataFrame back to the CSV file, overwriting the original file
data.to_csv(file_path, index=False)

# Display the value counts of 'annotation'
data['annotation'].value_counts()


Drop minority labels and save to a new csv file

In [ ]:
# import pandas as pd

# # Load the original CSV file
# input_csv = 'Compiled_ALL_Cryptocurrency.csv'
# df = pd.read_csv(input_csv)

# # Define a list of terms to be dropped
# terms_to_drop = [
#     'White Paper',
#     'Hard Fork',
#     'Proof-of-Concept',
#     'Minting',
#     'Fiat Currency',
#     'Airdrop',
#     'Money Laundering',
#     'TradFi',
#     'Multichain',
#     'Proof-of-Stake',
#     'Proof-of-Work',
#     'Crypto Winter',
#     'Proof-of-Reserves'
# ]

# # Filter the rows where 'annotation' does not contain any of the specified terms
# df_filtered = df[~df['annotation'].isin(terms_to_drop)]

# # Save the filtered data to a new CSV file
# output_csv = 'filtered_data.csv'
# df_filtered.to_csv(output_csv, index=False)
# df_filtered = data
# print(f"Filtered data saved to {output_csv}")


Testing the augmented models can use anot or logic anot / Provided example text

In [ ]:
import nlpaug.augmenter.word as naw

aug = naw.ContextualWordEmbsAug(model_path='distilroberta-base', action="substitute", device='cuda')

aug_sub = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute", device='cuda')

aug_sub2 = naw.ContextualWordEmbsAug(model_path='roberta-base', action="insert", device='cuda')

# Original sentence
original_sentence = "For space id, the tokens going toward the ecosystem fund, community airdrop, and the foundation will likely not be sold immediately for a profit."

# Augmenting using substitute action
augmented_sentence_substitute1= aug.augment(original_sentence)

# Augmenting using insert action
augmented_sentence_substitute2 = aug_sub.augment(original_sentence)

# Augmenting using insert action
augmented_sentence_substitute3 = aug_sub2.augment(original_sentence)

print("Substitute Augmented Sentence:")
print(augmented_sentence_substitute1)
print("\Substitute Augmented Sentence:")
print(augmented_sentence_substitute2)
print("\Substitute Augmented Sentence:")
print(augmented_sentence_substitute3)


Augmented the minority labels of each data for 3 times as using 3 diff models then save to augmented_data csv

In [ ]:
import pandas as pd
import nlpaug.augmenter.word as naw
from tqdm import tqdm
import random
import os
import nltk
os.environ["TOKENIZERS_PARALLELISM"] = "false"# Load your dataset
data = pd.read_csv(file_path)

# Define the minority labels and their respective counts
minority_labels = {
    "Money Laundering": 177,
    "Multichain": 134,
    "Proof-of-Concept": 109,
    "Fiat Currency": 108,
    "Proof-of-Stake": 52,
    "Proof-of-Reserves": 29,
    "Proof-of-Work": 21,
}

# Initialize the augmenter
aug = naw.ContextualWordEmbsAug(
    model_path='distilroberta-base', action="substitute", device='cuda')

back_translation_aug = naw.BackTranslationAug(
    from_model_name='Helsinki-NLP/opus-mt-en-zh',
    to_model_name='Helsinki-NLP/opus-mt-zh-en', device='cuda')

aug_synonym = naw.SynonymAug(aug_src='wordnet')

# Create lists to store augmented data
augmented_summaries = []
multilabels = []

for _, row in tqdm(data.iterrows(), total=len(data)):
    if isinstance(row['annotation'], str):
        augmented_labels = row['annotation'].split(',')  # Assuming labels are separated by commas
        augmented_labels_filtered = [label.strip() for label in augmented_labels if label.strip() in minority_labels]
        
        if augmented_labels_filtered:
            # Augment the summary for this row
            augmented_summary = aug.augment(row['text'])
            augmented_summary_backtranslate = back_translation_aug.augment(row['text'])
            augmented_summary_synonym = aug_synonym.augment(row['text'])
            # Append the augmented summary and corresponding multilabel to the lists
         
            augmented_summaries.append(augmented_summary)
            augmented_summaries.append(augmented_summary_backtranslate)
            augmented_summaries.append(augmented_summary_synonym)
            multilabels.append(row['annotation'])
            multilabels.append(row['annotation'])
            multilabels.append(row['annotation'])

    # Create a new DataFrame with Augmented_Summary and Multilabel columns
    augmented_df = pd.DataFrame({'text': augmented_summaries, 'annotation': multilabels})

# Save the augmented DataFrame to a new CSV file
data_augmented_path = 'augmented_data.csv'
augmented_df.to_csv(data_augmented_path, index=False)

Check total had augmented how many data for the minority labels

In [ ]:
data_augmented_path = "augmented_data.csv"
augmented_df = pd.read_csv(data_augmented_path)
augmented_df['annotation'].value_counts()

If the augmented data labels still lesser, pick those lesser and augmented 2 more times as using 2 diff models and save to augmented2_data csv

In [ ]:
import pandas as pd
import nlpaug.augmenter.word as naw
from tqdm import tqdm
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Load your dataset
data = pd.read_csv(file_path)

# Define the minority labels and their respective counts
minority_labels = {
    "Proof-of-Stake": 52,
    "Proof-of-Reserves": 29,
    "Proof-of-Work": 21,
}

# Initialize the augmenter
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute", device='cuda')

aug_insert = naw.ContextualWordEmbsAug(
    model_path='roberta-base', action="insert", device='cuda')

# Create lists to store augmented data
augmented_summaries = []
multilabels = []

for _, row in tqdm(data.iterrows(), total=len(data)):
    if isinstance(row['annotation'], str):
        augmented_labels = row['annotation'].split(',')  # Assuming labels are separated by commas
        augmented_labels_filtered = [label.strip() for label in augmented_labels if label.strip() in minority_labels]

        if augmented_labels_filtered:
            # Augment the summary for this row
            augmented_summary = aug.augment(row['text'])
            augmented_summary_insert = aug_insert.augment(row['text'])
            # Append the augmented summary and corresponding multilabel to the lists
            augmented_summaries.append(augmented_summary)
            augmented_summaries.append(augmented_summary_insert)
            multilabels.append(row['annotation'])
            multilabels.append(row['annotation'])

# Create a new DataFrame with Augmented_Summary and Multilabel columns
augmented2_df = pd.DataFrame({'text': augmented_summaries, 'annotation': multilabels})

# Save the augmented DataFrame to a new CSV file
data_augmented2_path = 'augmented2_data.csv'
augmented2_df.to_csv(data_augmented2_path, index=False)


Double check the minority labels had hit target anot, if still not then just proceed

In [ ]:
data_augmented2_path = "augmented2_data.csv"
augmented2_df = pd.read_csv(data_augmented2_path)
augmented2_df['annotation'].value_counts()

Combine the augmented_data and augmented2_data into augmented_final data csv

In [ ]:
import pandas as pd

# List of CSV file names to combine
csv_files = ['augmented_data.csv', 'augmented2_data.csv']

# Initialize an empty list to store the DataFrames
dfs = []

# Iterate over each CSV file and append its data to the list
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate the DataFrames in the list
combined_data = pd.concat(dfs, ignore_index=True)

# Sort the combined data based on the "annotation" column
sorted_data = combined_data.sort_values(by='annotation')

# Save the sorted data to a new CSV file
final_aug_path = "augmented_final.csv"
sorted_data.to_csv(final_aug_path, index=False)

sorted_data['annotation'].value_counts()


Split the original dataset into 3 sets (Training Validating and Testing)

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Suppose 'data' is your original dataframe

# Split the data into train and temp sets
train_data, temp_data = train_test_split(data, test_size=0.1, random_state=42)

# Save the 90% training dataset to a new CSV file
train_path = "training_data.csv"
train_data.to_csv(train_path, index=False)

# Split the temp data into test and validation sets
test_data, val_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Save the test dataset to a new CSV file
test_path = "test_data.csv"
test_data.to_csv(test_path, index=False)

# Save the validation dataset to a new CSV file
val_path = "validation_data.csv"
val_data.to_csv(val_path, index=False)

# Retrieve the value counts of the 'annotation' column in the train_data dataframe
value_counts = train_data['annotation'].value_counts()

# Print the value counts
print(value_counts)


Combine the training data and augmented_final data into train_data_augmented data csv

In [ ]:
import pandas as pd

# List of CSV file names to combine
csv_files = ['training_data.csv', 'augmented_final.csv']

# Initialize an empty list to store the DataFrames
dfs = []

# Iterate over each CSV file and append its data to the list
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate the DataFrames in the list
combined_data = pd.concat(dfs, ignore_index=True)

# Sort the combined data based on the "annotation" column
sorted2_data = combined_data.sort_values(by='annotation')

# Save the sorted data to a new CSV file
final_aug_path = "train_data_augmented.csv"
sorted2_data.to_csv(final_aug_path, index=False)

sorted2_data['annotation'].value_counts()

Downsampling the train_data_augmented csv into all data having equal labels and save to a new csv final_train_data_augmented data csv

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('train_data_augmented.csv')  # Replace 'path_to_your_file.csv' with the actual path to your CSV file

# Define the range for the random selection for each annotation
lower_limit = 100
upper_limit = 200

# Group the data by annotation and perform downsampling within each group
downsampled_data = df.groupby('annotation').apply(lambda x: x.sample(n=min(upper_limit, max(lower_limit, len(x))), random_state=42))

# Reset the index of the downsampled data
downsampled_data = downsampled_data.reset_index(drop=True)

# Write the downsampled data to a new CSV file
downsampled_data.to_csv('final_train_data_augmented.csv', index=False)  # The 'index=False' parameter is used to not write row indices

# Print the downsampled data
downsampled_data['annotation'].value_counts()

Remove brackets single quotes and double quotes on the text data

In [ ]:
import pandas as pd

# Replace 'data.csv' with the path to your CSV file
input_file = 'final_train_data_augmented.csv'
output_file = 'final_train_data_augmented.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Clean the 'text' column by removing square brackets, single quotes, and double quotes
df['text'] = df['text'].str.replace(r"[\[\]\"']", '', regex=True)

# Save the cleaned data to a new CSV file
df.to_csv(output_file, index=False)
